# Chainerに用意されていないデータセットを使ってみよう

ここでは、Chainerに予め用意されていないデータセットを外部から調達して、Chainerで記述されたネットワークの訓練のために用いる方法を具体例とともに示します。基本的な手順はほぼ前節のCIFAR10データセットクラスを拡張する方法と変わりません。

ほとんど同様の内容を繰り返すことを避けるため、ここではChainerが用意するデータセットクラス用のユーティリティの一つである`get_cross_validation_datasets`を使い、5-foldクロスバリデーションを簡単に行う方法も合わせて説明してみます。

まずは、データセットをダウンロードしてきます。今回は、日本人では数少ないKaggle Grand Masterであらせられるnagadomiさんが[こちら](http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/)で配布されているアニメキャラクターの顔領域サムネイルデータセットを使用します。

In [1]:
%%bash
if [ ! -f animeface-character-dataset.zip ]; then
    curl -O http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/data/animeface-character-dataset.zip
fi
if [ ! -d animeface-character-dataset ]; then
    unzip animeface-character-dataset.zip
fi
ls animeface-character-dataset

index.html
README.html
README.txt
thumb
tools


In [2]:
%%bash
if [ ! -f illust2vec_ver200.caffemodel ]; then
    curl -O http://illustration2vec.net/models/illust2vec_ver200.caffemodel
fi
if [ ! -f image_mean.npy ]; then
    curl -O http://illustration2vec.net/models/image_mean.npy
fi
ls illust2vec_ver200.caffemodel

illust2vec_ver200.caffemodel


今回はこちらのデータセットに含まれる画像を使って、キャラクターの顔画像を各キャラクターを表すクラスに分類してみます。ここでは、PILというパッケージを使用しますので、予めこれをインストールしておいてください。インストールは簡単で、pipを用いて以下のようにインストールできます。

```
% pip install PIL>=3.0
```

今回はバージョン3.0以上を想定しているため、バージョン指定を行っていますが、これをしない場合は自動的に最新版が入ります。

In [3]:
import glob
import os

import numpy as np

from PIL import Image
from chainer import dataset
from chainer.datasets import sub_dataset


class AnimeFaceDataset(dataset.DatasetMixin):
    
    IMG_DIR = 'animeface-character-dataset/thumb'
    IMG_SIZE = (160, 160)
    MEAN_FILE = 'image_mean.npy'
    
    def __init__(self):
        
        # thumbディレクトリ以下のディレクトリを列挙します。
        img_dirs = [d for d in glob.glob('{}/*'.format(self.IMG_DIR))
                    if os.path.isdir(d)]
        
        # 画像ファイルのパスを格納する空のリストを定義しておきます。
        self.img_fns = []
        
        for dname in img_dirs:
            
            # ignoreファイルが置かれているディレクトリは空か、
            # "分類の難易度が高い" (データセット配布元のREADME参照)
            # ため、スキップします。
            
            if len(glob.glob('{}/ignore'.format(dname))):
                continue
            
            # 画像ファイルへのパスのリストを追加していきます。
            self.img_fns += glob.glob('{}/*.png'.format(dname))
        
        # 各画像に対応するクラスIDをディレクトリ名から決定するためのリストです。
        self.cls_labels = list(set(os.path.dirname(fn)
                                   for fn in self.img_fns))
        
        self.mean = np.load(self.MEAN_FILE)
        self.mean = self.mean.mean(axis=(1, 2))
    
    def __len__(self):
        return len(self.img_fns)
    
    def get_example(self, i):
        
        # PILを使って画像を開き、画像サイズを揃えるためresizeします。
        img = Image.open(self.img_fns[i])
        img = img.resize(self.IMG_SIZE, Image.BICUBIC)
        img = np.asarray(img, dtype=np.float)
        if img.shape[0] > 3:
            img = img[:, :, :3]  # move alpha channel
        img = img.transpose(2, 0, 1)[::-1, ...]  # RGB -> BGR
        img -= self.mean[:, None, None]
        img = img.astype(np.float32)
            
        # 同一ディレクトリ名から同一のクラスラベルIDが引かれるようにします。
        dname = os.path.dirname(self.img_fns[i])
        label = np.asarray(self.cls_labels.index(dname), dtype=np.int32)
        
        return img, label

In [4]:
d = AnimeFaceDataset()
ds = sub_dataset.get_cross_validation_datasets_random(d, 5, seed=0)

In [5]:
import pickle

import chainer
import chainer.links as L
import chainer.functions as F

from chainer import Chain
from chainer.links.caffe import CaffeFunction
from chainer import serializers

n_classes = len(d.cls_labels)

class Illust2Vec(Chain):

    CAFFEMODEL_FN = 'illust2vec_ver200.caffemodel'
    PKL_FN = 'illust2vec_ver200.pkl'
    
    def __init__(self, n_classes):
        w = chainer.initializers.HeNormal()
        
        # 変換済みのChainerモデル（PKLファイル）が無い場合
        if not os.path.exists(self.PKL_FN):
            # CaffeModelを読み込んで保存（時間がかかります）
            model = CaffeFunction(self.CAFFEMODEL_FN)
            pickle.dump(model, open(self.PKL_FN, 'wb'))
        else:
            model = pickle.load(open(self.PKL_FN, 'rb'))
        del model.encode1
        del model.forwards['encode1']
        model._children.pop()
        model._children.pop()
        model.layers = model.layers[:-2]
        super(Illust2Vec, self).__init__(
            trunk=model,
            fc6=L.Linear(None, 4096, initialW=w),
            fc7=L.Linear(4096, 4096, initialW=w),
            fc8=L.Linear(4096, n_classes, initialW=w))
        self.train = True

    def __call__(self, x):
        h = self.trunk({'data': x}, ['conv6_3'], train=self.train)[0]
        h.creator = None
        h = F.dropout(F.relu(self.fc6(h)), train=self.train)
        h = F.dropout(F.relu(self.fc7(h)), train=self.train)
        return self.fc8(h)
        
model = Illust2Vec(n_classes)
model = L.Classifier(model)

In [6]:
from chainer import iterators
from chainer import training
from chainer import optimizers
from chainer.training import extensions
from chainer.dataset import concat_examples

batchsize = 16
gpu_id = 0

class TestModeEvaluator(extensions.Evaluator):
    
    def evaluate(self):
        model = self.get_target('main')
        model.train = False
        ret = super(TestModeEvaluatr, self).evaluate()
        model.train = True
        return ret
        

for split_i in range(len(ds)):
    train, valid = ds[split_i]
    train_iter = iterators.SerialIterator(train, batchsize)
    valid_iter = iterators.SerialIterator(valid, batchsize,
                                          repeat=False, shuffle=False)
    
    optimizer = optimizers.Adam()
    optimizer.setup(model)
    
    updater = training.StandardUpdater(
        train_iter, optimizer, device=gpu_id)
    
    trainer = training.Trainer(updater, (100, 'epoch'))
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.PrintReport(
        ['epoch', 'iteration', 'main/loss', 'main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.PlotReport(['main/loss'], file_name='loss.png')
    trainer.extend(extensions.PlotReport(['main/accuracy'], file_name='accuracy.png')
    trainer.extend(TestModeEvaluator(valid_iter, model, device=gpu_id))

    trainer.run()

epoch       iteration   main/loss   main/accuracy  elapsed_time
0           10          59.6912     0.03125        2.80055       
0           20          51.2553     0.01875        4.01365       
0           30          23.2628     0.0875         5.22878       
0           40          13.2628     0.14375        6.43622       
0           50          7.28875     0.23125        7.64638       
0           60          6.9367      0.21875        8.86511       
0           70          5.51124     0.29375        10.0796       
0           80          4.30844     0.3375         11.2971       
0           90          5.31356     0.30625        12.5096       
0           100         6.46435     0.29375        13.7267       
0           110         5.33596     0.375          14.945        
0           120         5.51989     0.3375         16.1597       
0           130         5.52243     0.36875        17.374        
0           140         5.1177      0.40625        18.5895       
0           

KeyboardInterrupt: 

In [ ]:
ds[0]

In [ ]:
len(ds[0])

In [ ]:
a, b = ds[0]

In [ ]:
len(a)